#### Applied Machine Learning - Mini Project 2 (Tasnim Ahmed, ta1743)

# Decision Trees

In [ ]:
# node - feature attribute (does the item meet the specific category)
# branches - decision (rule - yes or no)
# leaf - an outcome (the category - leaf name or spam)

In [1]:
import numpy as np 
import sklearn 
import pandas as pd
import math

Given the fact both of the datasets have continuous features we will implement decision trees that have binary splits.

In [26]:
class Node: 
    def __init__(self, left = None, right = None, feature = None, threshold = None, label = None):
        self.left = left
        self.right = right
        self.feature = feature        # on which feature was the node divided upon 
        self.threshold = threshold    # on what threshold of the feature was it divided upon
        
        self.label = label            # place the label if it is a tree node, else none
        
        def check_leaf_node(self):
            return self.label != None
        
        def label_value(self):        # return the label value if it is a leaf node
            return self.label
            
        
class DecisionTreeClassifier:
    def __init__(self, n_min, n_features):
        self.root = None                # the root of the node
        self.n_min = n_min              # the stopping criteria
        
        
    def fit(self, X, y):
        #set the stopping hyperparameter
        min_sample_split = X.shape[0]*(self.n_min/100)
        self.root = self.grow_tree(X, y, min_sample_split)
        
    def grow_tree(self, X, y, min_sample_split):    #recursive function
        sample_size, n_features = X.shape
        labels = np.unique(y)
        n_labels = len(labels)
        
        # checking the stopping criteria 
        if (n_samples <= min_sample_split or n_labels == 1):
            #find the max label of the dataset and set that as the leaf node value
            leaf_node = Node(label = max_label(y))
            return leaf_node   #return?
           
        # get the best feature split and threshold for the given dataset
        split_feature, split_threshold = find_best_split(X, y, n_features)
        
        # create child nodes
        # call the grow tree function again on the subset of the data
        
        return root
      
    # find the best split (threshold and feature) upon which the given dataset will be separated
    def find_best_split(self, X, y):
        
        n_features = X.shape[1]
        col_ig_dict = {}
        
        # iterate over the columnns 
        for i in range(n_features):
            #get the max ig value and threshold value for each column and store in the dict
            #INCOMPLETE
            #send column X[, i] to col_information_gain function
            #store those values in the dict
            col_ig_dict[i] = 0
    
        #the feature with highest information gain is selected as the best one 
        best_threshold = max(col_ig_dict, key=col_ig.get)
        best_feature = col_ig_dict.get(best_threshold)
                
        return best_feature, best_threshold
        
    
    def max_label(self, y):
        labels = np.unique(y)
        n_labels = len(labels)
        label_dict = {}
        
        for i in range(n_labels):
            label_dict[labels[i]] = 0
        
        for i in range(n_labels):
            for j in range(len(y)):
                if y[j] == labels[i]:
                    label_dict[labels[i]] += 1
                    
        #find the max label and return that    
        label = max(label_dict, key = label_dict.get)
        return label
    
    def entropy(self, column):
        counts = np.bincount(column)
        entropy = 0
        for count in counts:
            if (count/len(column)) > 0:
                entropy += (count/len(column))*math.log(count/len(column, 2))                               
        return entropy  
    
    
    def information_gain(self, X_col, y, threshold):
        #calculate the original entropy of the target before split
        main_entropy = entropy(y)
        
        #Split the data based on the threshold  
        left_split = left_target = right_split = right_target = []
        for i in range(len(X_col)):
            # get the rows less than the threshold
            if X_col[i] < threshold:
                left_split.append(X_col[i])
                left_target.append(y[i])
            # get the rows more than or equal to the threshold
            else:
                right_split.append(X_col[i])
                right_target.append(y[i])
    
        splits = [left_split, right_split]
        target_splits = [left_target, right_target]
        to_sub = 0
        
        for i in range(len(splits)):
            prob = len(splits[i])/len(X_col)
            to_sub += prob*entropy(target_splits[i])
                                     
        return main_entropy - sub
        
    def col_information_gain(self, X_col, y):
        # dictionary that stores the theshold values with the information gain {threshold: ig}
        col_igs = {}
        
        # iterate over the rows
        for i in range(len(X_col) - 1):
            # calculate the average of adjacent row values
            avg_thr = (X_col[i] + X_col[i+1])/2
            # get the information gain and add it to the dictionary 
            col_igs[avg_thr] = information_gain(X_col, y, avg_thr)
        
        max_threshold = max(col_ig, key=col_ig.get)
        max_ig = col_igs.get(max_threshold)
                                                    
        return {max_ig, max_threshold}
    
    def predict_y(self, X, y):
        pass

In [24]:
arr1 = [1,0,1,1,1,0, 2]

## Iris 

In [1]:
# Has three class - label them as yes. no or maybe
# predict the classes given 4 features - length and width of sepals and petals
# 150 instance, each class having 50 instances

In [6]:
# Loading the data
iris_data  = pd.read_csv("iris.csv", names = ["f1", "f2", "f3", "f4", "target"])

In [7]:
# Verifying if the data has been loaded properly
iris_data.head()

,f1,f2,f3,f4,target
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [8]:
# Checking the shape of the data
iris_data.shape

(150, 5)

In [15]:
# Separating the features and the target variable columns
X = iris_data.drop('target', axis = 'columns') #independent variables
y = iris_data['target']  #dependent variable

rows, cols = X.shape
print(X.columns.values)
print(y.shape)



['f1' 'f2' 'f3' 'f4']
(150,)


KeyError: 1

In [ ]:
# set the species values to 0, 1 and 2

In [ ]:
#fit the model

In [ ]:
# check the accuracy with kfold cross validation

## Spambase

In [ ]:
## Load the data

In [ ]:
# run through thr decision tree with kfold and n_min array

In [ ]:
# check the accuracy